# Project 2: Build Decision Tree for Attrition Rate Analysis
DV - "Attrition"
IDV - Output of RF Algorithm

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import preprocessing

attrition_data = pd.read_csv('general_data.csv')
label_encoder = preprocessing.LabelEncoder()

attrition_data.dropna()
attrition_data.isnull()

meanof_totworkingyrs = attrition_data['TotalWorkingYears'].mean()
attrition_data['TotalWorkingYears'] = np.where(attrition_data['TotalWorkingYears'].isnull(),meanof_totworkingyrs,attrition_data['TotalWorkingYears'])

meanof_NumCompaniesWorked = attrition_data['NumCompaniesWorked'].mean()
attrition_data['NumCompaniesWorked'] = np.where(attrition_data['NumCompaniesWorked'].isnull(),meanof_NumCompaniesWorked,attrition_data['NumCompaniesWorked'])

attrition_data['BusinessTravel'] = label_encoder.fit_transform(attrition_data['BusinessTravel'])
attrition_data['Department'] = label_encoder.fit_transform(attrition_data['Department'])
attrition_data['EducationField'] = label_encoder.fit_transform(attrition_data['EducationField'])
attrition_data['Gender'] = label_encoder.fit_transform(attrition_data['Gender'])
attrition_data['JobRole'] = label_encoder.fit_transform(attrition_data['JobRole'])
attrition_data['MaritalStatus'] = label_encoder.fit_transform(attrition_data['MaritalStatus'])
attrition_data['Attrition'] = label_encoder.fit_transform(attrition_data['Attrition'])

rf_model = RandomForestClassifier(n_estimators=1000, max_features=3, oob_score=True)

#print(attrition_data.columns)

features = ['Age','BusinessTravel','DistanceFromHome','Education','EducationField','EmployeeID','Gender','JobLevel','JobRole','MaritalStatus','MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','StandardHours','StockOptionLevel','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsSinceLastPromotion','YearsWithCurrManager']

rf_model.fit(X=attrition_data[features],y=attrition_data['Attrition'])

print("OOB Accuracy:",rf_model.oob_score)

for features,imp in zip(features,rf_model.feature_importances_):
    print(features,imp)


OOB Accuracy: True
Age 0.09906290558307419
BusinessTravel 0.027123740598931072
DistanceFromHome 0.07046176842037659
Education 0.03882399415045388
EducationField 0.0433903761506409
EmployeeID 0.03287294689903609
Gender 0.01657081301708936
JobLevel 0.035331263260994575
JobRole 0.05512871139615008
MaritalStatus 0.03900785809896242
MonthlyIncome 0.10071631116846773
NumCompaniesWorked 0.055531301690373853
PercentSalaryHike 0.06579098672585885
StandardHours 0.0
StockOptionLevel 0.03320211899267339
TotalWorkingYears 0.08521385414976847
TrainingTimesLastYear 0.04348467203840119
YearsAtCompany 0.06499771070983398
YearsSinceLastPromotion 0.0421588536391876
YearsWithCurrManager 0.0511298133097258


In [18]:
predictors = pd.DataFrame([attrition_data['Age'],attrition_data['DistanceFromHome'],attrition_data['JobRole'],attrition_data['MonthlyIncome'],attrition_data['PercentSalaryHike'],attrition_data['TotalWorkingYears'], attrition_data['NumCompaniesWorked'],attrition_data['YearsAtCompany']]).T

tree_model = tree.DecisionTreeClassifier(max_depth=8)

tree_model.fit(X=predictors,y=attrition_data['Attrition'])

with open('Dtree_attrition.dot','w') as f:
    f=tree.export_graphviz(tree_model,feature_names=['Age','DistanceFromHome','JobRole','MonthlyIncome','PercentSalaryHike','TotalWorkingYears','NumCompaniesWorked','YearsAtCompany'],out_file=f)
    
accuracy = tree_model.score(X=predictors,y=attrition_data['Attrition'])

print(accuracy*100,"% Accurate model we got!")


89.97732426303855 % Accurate model we got!


# Rules based on Decision tree model
1. if 'PercentSalaryHike <= 20.0' and 'DistanceFromHome <= 4.0' and 'Age <= 18.5' and 'MonthlyIncome <= 112610.0' and 'TotalWorkingYears <= 1.5' and 'Age <= 33.5' then 100% employee left from company (100% Attrition yes).
2. if 'MonthlyIncome <= 106255.0' and Age > 18.5 & <=23.5' and 'MonthlyIncome > 23140.0'  and 'TotalWorkingYears <= 1.5' then 100% employee left from company (100% Attrition yes).

3. if 'PercentSalaryHike <= 13.5' and 'Age <= 32.0' and 'PercentSalaryHike <= 21.0' and 'YearsAtCompany <= 1.5' and 'TotalWorkingYears > 1.5' then 100% employee left from company (100% Attrition yes).
4. if 'MonthlyIncome > 176015.0' and 'Age <= 33.5' and 'NumCompaniesWorked <= 4.5' and 'YearsAtCompany > 1.5' and 'TotalWorkingYears > 1.5 and <=5.0' then 100% employee left from company (100% Attrition yes).
